In [212]:
import numpy as np
from numpy import array,newaxis
from numpy import sqrt,sin, pi, exp
import pandas as pd
import abc
from abc import abstractmethod,ABCMeta

In [213]:
class UniformGrid(metaclass=ABCMeta):
    def __init__(self,Xm, Xn,N):#輸入三個參數:Xm,Xn,N，得到四個參數:Xm,Xn,N,h
        self.Xm=Xm
        self.Xn=Xn
        self.N=N
        self.h=(Xm-Xn)/(N-1)
    def grid(self):#做出網格的方法
        (Xm,Xn,N,h)=(self.Xm,self.Xn,self.N,self.h)
        a=[]
        for i in range(self.N):
            a.append(Xn+i*h)
        return a
    @abstractmethod #確保solver一定要能執行
    def solver(self):
        pass

In [214]:
class C_N_Solver(UniformGrid):
    def __init__(self,Xm,Xn,N,D,dt,t,f,u1,u2,u=[]):#u不輸入的話，則自動設為空的list
        UniformGrid.__init__(self,Xm,Xn,N)
        self.h=(Xm-Xn)/(N-1)
        self.D=D
        self.dt=dt
        self.t=t
        self.f=f
        self.u1=u1
        self.u2=u2
        self.u=u
    def solver(self):  
        (h,dt,D,N,f,u1,u2,Xm,Xn,t,u)=(self.h,self.dt,self.D,self.N,self.f,self.u1,self.u2,self.Xm,self.Xn,self.t,self.u)
        if any(u)==False: #判斷是否有給初始矩陣，如果已經算過一次就有，還沒算過則無
            z=[]
            for ele in super().grid():
                z.append(f(ele,t))
            u=array(z)[:,newaxis]
        #a為左邊的矩陣
        s = (N,N)
        a=np.zeros(s)
        a[0,0]=1
        a[N-1,N-1]=1
        for i in range(1,N-1):
            a[i,i-1]=-D*dt/2/h**2
            a[i,i+1]=-D*dt/2/h**2
            a[i,i]=D*dt/h**2+1
        #b為右邊的矩陣
        q = (N,1) 
        b=np.zeros(q)
        c=D*dt/2/h**2
        for i in range(1,N-1):
            b[i]=u[i]+c*(u[i-1]-2*u[i]+u[i+1])
        #e為解出來的答案
        e=np.linalg.solve(a,b)
        e[0,0]=u1 #帶入boundary condition u1
        e[N-1,0]=u2 #帶入boundary condition u2
        self.u=e #將u指定為這次的解，以便於進行下次的疊帶運算
        return e
    def output(self):#設定輸出
        (h,dt,D,N,f,u1,u2,Xm,Xn,t,u)=(self.h,self.dt,self.D,self.N,self.f,self.u1,self.u2,self.Xm,self.Xn,self.t,self.u)
        def time(t,dt,N):
            c=[]
            for i in range(N):
                c.append(t+dt)
            return c
        a=time(t,dt,N)#time的list
        b=super().grid()#space的list
        c=list(u.ravel())#solution的list，ravel為降階用
        Total = np.array([b,a,c]).T
        df=pd.DataFrame( data = Total)#轉成dataframe以利輸出
        df.to_csv(path_or_buf='third practice.csv',encoding='’utf_8_sig' ,mode='a',header=False)

In [224]:
#測試題目的輸出
def s(x,t): #sin(pi*x)輸入的矩陣 N*1
    return sin(pi*x)
t=0
dt=0.001
A=C_N_Solver(1,0,51,1,dt,t,s,0,0)
#A.solver()
for i in range(9):
    u=A.solver() #要先將u算出來，才能帶入輸出的格式
    C_N_Solver(1,0,51,1,dt,t,s,0,0,u).output()
    t=t+dt #t也要改變成t+dt

In [223]:
#求出測試題目的答案
def s(x,t): #sin(pi*x)輸入的矩陣 N*1
    return sin(pi*x)
c=[]
t=0
dt=0.001
A=C_N_Solver(1,0,51,1,dt,t,s,0,0)
for i in range(9):
    c.append(A.solver())
c[-1]

array([[0.        ],
       [0.05745524],
       [0.11468372],
       [0.17145961],
       [0.22755882],
       [0.28275996],
       [0.33684517],
       [0.38960101],
       [0.44081928],
       [0.49029783],
       [0.5378414 ],
       [0.58326236],
       [0.62638144],
       [0.66702849],
       [0.70504308],
       [0.74027518],
       [0.77258576],
       [0.80184729],
       [0.8279443 ],
       [0.85077379],
       [0.87024567],
       [0.88628308],
       [0.89882274],
       [0.90781516],
       [0.91322484],
       [0.91503044],
       [0.91322484],
       [0.90781516],
       [0.89882274],
       [0.88628308],
       [0.87024567],
       [0.85077379],
       [0.8279443 ],
       [0.80184729],
       [0.77258576],
       [0.74027518],
       [0.70504308],
       [0.66702849],
       [0.62638144],
       [0.58326236],
       [0.5378414 ],
       [0.49029783],
       [0.44081928],
       [0.38960101],
       [0.33684517],
       [0.28275996],
       [0.22755882],
       [0.171